# Fast low-rank nonnegative tensor factorizations in tensor train format

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import scipy.io as spio
from time import time

from tensor_train import TensorTrain

In [2]:
## TTSVD, GetFullTensor

sizes = [10, 20, 20]
ranks = [1, 10, 10, 1]

tt = TensorTrain(sizes=sizes, ranks=ranks, seed=42)
full = TensorTrain.GetFullTensor(tt.GetCores())
np.linalg.norm(full - TensorTrain.GetFullTensor(TensorTrain.TTSVD(full, [1, 10, 10, 1]))) / np.linalg.norm(full)

8.952125031870687e-15

In [3]:
## DotProduct

tt1 = TensorTrain(sizes=sizes, ranks=ranks)
full1 = TensorTrain.GetFullTensor(tt1.GetCores())

tt2 = TensorTrain(sizes=sizes, ranks=ranks)
full2 = TensorTrain.GetFullTensor(tt2.GetCores())

print(TensorTrain.DotProduct(tt1, tt2) - (full1 * full2).sum())

0.0


In [4]:
## Sum


tt1 = TensorTrain(sizes=sizes, ranks=ranks)
full1 = TensorTrain.GetFullTensor(tt1.GetCores())

tt2 = TensorTrain(sizes=sizes, ranks=ranks)
full2 = TensorTrain.GetFullTensor(tt2.GetCores())

print(np.linalg.norm(full1 + full2 - TensorTrain.GetFullTensor((tt1 + tt2).GetCores())))

2.313700521422292e-13


In [5]:
## Hadamard product

tt1 = TensorTrain(sizes=sizes, ranks=ranks)
full1 = TensorTrain.GetFullTensor(tt1.GetCores())

tt2 = TensorTrain(sizes=sizes, ranks=ranks)
full2 = TensorTrain.GetFullTensor(tt2.GetCores())

print(np.linalg.norm(full1 * full2 - TensorTrain.GetFullTensor((tt1 * tt2).GetCores())))

3.0398277189160978e-12


In [6]:
## Orthogonalize

tt1 = TensorTrain(sizes=[5, 7, 8, 10], ranks=[1, 3, 3, 3, 1])
tt1.Orthogonalize()

t = tt1._cores[0].squeeze(0)
print(np.linalg.norm(t.T @ t - np.eye(t.shape[1])))
for p in range(1, len(tt1._cores) - 1):
    t = t @ tt1._cores[p].reshape(tt1._ranks[p], -1)
    t = t.reshape(t.shape[0] * tt1._sizes[p], tt1._ranks[p + 1])
    
    print(np.linalg.norm(t.T @ t - np.eye(t.shape[1])))

6.279429183987136e-16
8.167251954756424e-16
5.930017789846812e-16


In [7]:
## Compress

tt1 = TensorTrain(sizes=[5, 7, 8, 10], ranks=[1, 3, 3, 3, 1])
tt1.Compress(1e-6)

t = tt1._cores[3].squeeze(2)
print(np.linalg.norm(t @ t.T - np.eye(t.shape[0])))
for p in range(len(tt1._cores) - 2, 0, -1):
    t = tt1._cores[p].reshape(-1, tt1._ranks[p + 1]) @ t
    t = t.reshape(tt1._ranks[p], -1)
    
    print(np.linalg.norm(t @ t.T - np.eye(t.shape[0])))

1.4502115813784181e-15
2.270341705087961e-15
9.534353327576943e-16
